In [1]:
import sys, os
from tqdm import tqdm
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice")
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice\third_party\Matcha-TTS")

from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
from cosyvoice.utils.common import set_all_random_seed
import torchaudio

2025-02-18 14:53:34,889 - modelscope - INFO - PyTorch version 2.3.1+cu121 Found.
2025-02-18 14:53:34,894 - modelscope - INFO - Loading ast index from C:\Users\CHENHAOYUAN\.cache\modelscope\ast_indexer
2025-02-18 14:53:35,032 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 553a05bea23a95025342c8774cbd7108 and a total number of 980 components indexed


failed to import ttsfrd, use WeTextProcessing instead


## 加载模型

In [2]:
cosyvoice = CosyVoice2(R'D:\GITHUB\FunAudioLLM-APP\cosyvoice\pretrained_models\CosyVoice2-0.5B', load_jit=False, load_trt=False, fp16=False)

d:\anaconda3\envs\cosyvoice\lib\site-packages\diffusers\models\lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-02-18 14:53:49,458 INFO input frame rate=25
d:\anaconda3\envs\cosyvoice\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-02-18 14:53:51,280 WETEXT INFO building fst for zh_normalizer ...
2025-02-18 14:53:51,280 INFO building fst for zh_normalizer ...
2025-02-18 14:54:13,830 WETEXT INFO done
2025-02-18 14:54:13,830 INFO done
2025-02-18 14:54:13,831 WETEXT INFO fst path: d:\anaconda3\envs\cosyvoice\lib\site-packages\tn\zh_tn_tagger.fst
202

## 加载音频
需要手动标记一下音频的内容

In [3]:
prompt_audio = load_wav(R'D:\GITHUB\OpenVoice\inputs\train\tvboy1_denoised.wav', 16000)
prompt_text = "两个或者三个吧，嗯肚子很胀，就是，有点，嗯，有时候会吐的"

## 配置

In [4]:
batch = 4 # doesn't work at this moment
output_dir = "./tvboy_denoised_mandarin_seed34751218"
seed = 34751218

# 以下选一个

## 普通话
input_txts = "./mandarin.txt"
cross_lang = ""

## 粤语
# input_txts = "./cantonese.txt"
# cross_lang = "粤语"

# ！！下面的不要动

if not os.path.exists(output_dir):
	os.makedirs(output_dir, exist_ok=True)
with open(input_txts, 'r', encoding='utf-8') as f:
	input_text_lines = [ x.strip() for x in f.readlines() if x.strip() != "" ]
	print("Got dataset: ", len(input_text_lines))
	def input_generator():
		for i in range(0, len(input_text_lines), batch):
			yield input_text_lines[i:min(i+batch, len(input_text_lines))]
set_all_random_seed(seed)
print("Output folder: ", output_dir)
print("Using seed: ", seed)

Got dataset:  418
Output folder:  ./tvboy_denoised_mandarin_seed34751218
Using seed:  34751218


## 测试推理

In [61]:
sample_text = input_text_lines[0]
def zero_infer():
	print("Using zero infer.")
	ret = list(cosyvoice.inference_zero_shot(sample_text, prompt_text, prompt_audio, stream=False))[0]
	torchaudio.save("zero_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
def cross_infer():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	ret = list(cosyvoice.inference_instruct2(sample_text, instruct_text, prompt_audio, stream=False))[0]
	torchaudio.save("cross_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_infer()
else:
	cross_infer()
print("Done.")

Using zero infer.


  0%|          | 0/1 [00:00<?, ?it/s]2025-02-17 11:49:48,140 INFO synthesis text 他在家里吃苹果，喝茶，打电话，唱歌。
2025-02-17 11:49:53,008 INFO yield speech len 6.92, rtf 0.7034620453167513
100%|██████████| 1/1 [00:05<00:00,  5.28s/it]

Done.


## 生成

In [5]:
def zero_shot():
	print("Using zero shot.")
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_zero_shot(data, prompt_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
def cross_shot():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_instruct2(data, instruct_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_shot()
else:
	cross_shot()
print("Done.")

Using zero shot.


0it [00:00, ?it/s]

Skip  ./tvboy_denoised_mandarin_seed34751218\1.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\2.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\3.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\4.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\5.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\6.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\7.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\8.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\9.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\10.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\11.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\12.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\13.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\14.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\15.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\16.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\17.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\18.wav
Skip  ./tvboy_denoised_mandarin_seed34751218\19.wav
Skip  ./tvboy_denoise

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-18 14:54:18,137 INFO synthesis text 计算机科学的快速发展使得人工智能得到了广泛应用。
d:\anaconda3\envs\cosyvoice\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
2025-02-18 14:54:23,478 INFO yield speech len 6.32, rtf 0.8450118801261804
100%|██████████| 1/1 [00:05<00:00,  5.96s/it]


...

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-18 15:01:35,669 INFO synthesis text 餐厅里传来了愉快的谈话声和笑声。
2025-02-18 15:01:39,032 INFO yield speech len 4.84, rtf 0.6948771062961295
100%|██████████| 1/1 [00:03<00:00,  3.93s/it]


min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-18 15:01:39,609 INFO synthesis text 我们去山顶看日出，景色美得令人叹为观止。
2025-02-18 15:01:43,687 INFO yield speech len 6.0, rtf 0.6797613302866617
100%|██████████| 1/1 [00:04<00:00,  4.64s/it]
105it [07:26,  4.25s/it]

Done.
